In [11]:
import os

In [12]:
%pwd

'/Users/manan/Desktop/Coccidiosis-Classification/Coccidiosis-Disease-Classification/research'

In [13]:
os.chdir("../")

In [14]:
%pwd

'/Users/manan/Desktop/Coccidiosis-Classification/Coccidiosis-Disease-Classification'

In [10]:
import tensorflow as tf

In [16]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [15]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [17]:
from CnnClassifier.constants import *
from CnnClassifier.utils.common import read_yaml, create_directories, save_json

In [18]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            create_directories([self.config.artifacts_root])

    def get_validation_config(self) -> EvaluationConfig:
          eval_config = EvaluationConfig(
                path_of_model="artifacts/training/model.h5",
                training_data="artifacts/data_ingestion/Chicken-fecal-images",
                all_params=self.params,
                params_image_size=self.params.IMAGE_SIZE,
                params_batch_size=self.params.BATCH_SIZE
          )
          return eval_config


In [19]:
from urllib.parse import urlparse

In [20]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _vaild_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.30
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.vaild_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._vaild_generator()
        self.score = model.evaluate(self.vaild_generator)

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)


In [21]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()

except Exception as e:
    raise e

Found 116 images belonging to 2 classes.
8/8 [==============================] - 25s 3s/step - loss: 0.5828 - accuracy: 0.9483
